In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%pip install levenshtein

# Sequence-based similarity

Consider search engines that have indexes of text content. Entering a misspelled search term will match misspellings in the database, but modern search engines often suggest similar queries with larger number of hits in the database. How do they do it? Search engines (like elasticsearch) generate a wide range of possible misspellings of the query and present search results (or spelling suggestions) informed by queries for a comprehensive set of deletion- substitution-, insertion-, and transposition- modified words.

Edit distance (also known as Levenshtein distance) is another approach at fuzzy matching, inspired by the process of changing one input into another. It is more sensitive than Jaccard similarity scoring but requires more computations (that is to say, it takes more time to run when other factors are held constant).

## Hamming distance

If all of the symbols of interest are the same length, say, five-letter telegraph abbreviations, it is simple to count the number of differences letter-by-letter: ENBET differs in one position from ERBET and differs in two positions from TRBET. 

This counting-of-differences distance metric goes by the name ''Hamming distance'', but this is suited for vocabularies of aligned codewords all the same length.  

Hamming distance would make sense, for instance, comparing dates in ISO-8601 format represented as strings, like "2020-03-15" and "2023-09-21".  Differences of a single character have Hamming distance one; transpositions of digits have Hamming distance 2. 

## Levenshtein (edit) distance

If you think about the process of editing text, you can modify text by adding text, deleting text, or, if you are inclined, by both deleting and adding in about the same place. By counting the number of steps needed to transform `string1` into `string2` using these three operations, **deletion**, **insertion**, and **substitution**, we can define a number that describes how similar the two strings are.  There are algorithms that will find the minimum number of edits that will transform one string into another (similar) string and this minimum number of edits score has many uses in data munging.

The Levenshtein distance counts the differences between two strings by describing the minimum number of steps needed to transform one string into the other, where the permitted steps are
1. substitution (replace a character with a different one)
2. insertion (add a character), and
3. deletion.

This is the number of single-character edits needed to transform one string into another.


Levenshtein distance is fairly easy to illustrate by providing an example alignment: 

The first string can be changed into the second with a three-character deletion (`n't`), a three-character substitution (`she` for `you`), and eight characters of insertions  `w` and `e to go`. 

The edit distance between the strings is the sum of the number of changes.  

Search engines which index text can sometimes give reasonable results when given inexact search queries, but it is not without some engineering.  Entering a misspelled search term will match misspellings in the database and will fail to find exact matches on the correct spelling.  Modern search engines often suggest similar queries with larger number of hits in the database. How do they do it? Search engines (like elasticsearch) generate a wide range of possible misspellings of the query and present search results (or spelling suggestions) informed by queries for a comprehensive set of deletion- substitution-, insertion-, and transposition- modified words.  Yes, you have to check the search index hundreds of times, but if you see a one- or two-letter different search term with hundreds of times the hits, you would do well to suggest it.

In [ ]:
import Levenshtein

In [ ]:
from Levenshtein import distance

The `Levenshtein.distance` function finds the minimum number of steps.  For two completely different words, all the letters must be replaced:

In [ ]:
distance("one", "six")

While for a word that is a subset of another, the missing letters must all be added:

In [ ]:
distance("seven", "seventeen")

## Distances between number words

Here we calculate the Levenshtein distance between the words for the first twenty counting numbers in English:

In [ ]:
numbers2 = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
           "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen",
           "eighteen", "nineteen", "twenty"]

In [ ]:
d = np.zeros((20,20))
# Calculate distance between all pairs of strings in numbers2
for i in range(len(numbers2)):
    for j in range(len(numbers2)):
        d[i][j] = distance( numbers2[i], numbers2[j])


In [ ]:
plt.imshow(d)
plt.colorbar()
plt.title("Levenshtein distance between number words")
plt.yticks(np.arange(len(numbers2)), numbers2, fontsize=6);
plt.xticks(np.arange(len(numbers2)), numbers2, fontsize=6, rotation=90);

This looks different from the set-based number-word similarity graph we constructed earlier.  First, we are measuring distance, not similarity, so the diagonal has distance 0 instead of similarity 1.  Second, the range of the distances is integers between 0 and the sum of the lengths of the two strings being compared.  

Since this depends on a property of the strings that is not interesting, we can divide by the maximum possible 

The calculation of edit distance is, in general, more expensive to calculate than Jaccard similarity or Hamming distance; there are three possible operations at each position, and the operations may need to be repeated.


In [ ]:
d2 = np.zeros((20,20))
# Calculate distance between all pairs of strings in numbers2
for i in range(len(numbers2)):
    for j in range(len(numbers2)):
        d2[i][j] = distance( numbers2[i], numbers2[j]) / (len(numbers2[i]) + len(numbers2[j])) 

In [ ]:
plt.imshow(1-d2)
plt.colorbar()
plt.title("Levenshtein similarity between number words")
plt.yticks(np.arange(len(numbers2)), numbers2, fontsize=6);
plt.xticks(np.arange(len(numbers2)), numbers2, fontsize=6, rotation=90);

## Spelling correction

One straightforward application of fuzzy matching is spelling correction; after checking each token in a document to see if it is in the dictionary, the words which are not in the dictionary are presented to the user along with suggestions of dictionary words that are similar to the unmatched word.

The fuzzy matching can either start from the misspelled word and generate all possible misspellings out to a certain edit distance, or can start from the words in the dictionary, and find those with the fewest needed edits.  Think for a minute which of these might be cheaper (in terms of number of comparisons required.) 

In [ ]:
words = pd.read_csv("../../data/SINGLE.txt", header=None)

In [ ]:
words

In [ ]:
# Start with a special-purpose function that only finds the distance
# to one word of interest: 
word = "wprd"   # This is a misspelling
def dist_to_word(w):
        return distance(w, word)

In [ ]:
dist_to_word("worm"), dist_to_word("wierd"), dist_to_word("weird")

In [ ]:
# Check that d(word, word) == 0 
dist_to_word("word") == 0 

Check that we can find all the distances in one step:

In [ ]:
#note xmode Minimal shortens the error message
%xmode Minimal

In [ ]:
words[0].apply(dist_to_word)

Well, that didn't work.  It seems some of the words in the dictionary aren't encoded as strings.  Let us find the rows that are broken:

In [ ]:
def test_str(s):
    return type(s) == str
words[0].apply(test_str)

In [ ]:
badindexes = np.where(words[0].apply(test_str)==False)
badindexes

Three rows have non-string names in them.

In [ ]:
words.loc[badindexes]

The non-string rows are all encoded as `NaN` (Not-a-number, a special symbol that acts like a number).  Examining the dataframe does not tell us about the source of the problem.  Let's try specifying a data type to `read_csv` and hope for the best:

In [ ]:
words = pd.read_csv("../../data/SINGLE.txt", header=None, dtype="str")

In [ ]:
words.loc[badindexes]

That didn't work either, the bad values are still bad.  The documentation for `read_csv` https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
has a parameter called `na_filter` that can be turned off.

In [ ]:
words = pd.read_csv("../../data/SINGLE.txt", header=None, na_filter=False)

And now we test for bad values: 

In [ ]:
np.where(words[0].apply(test_str)==False)

We find none.  What were those bad values, now?

In [ ]:
words.loc[badindexes]

Huh.  Might want to take note of that in the future, some strings, by default, get turned into number-like NaNs.

Now let us find the distances between our word and all the other words:

In [ ]:
# define this function in a function that we can use on all the rows in 
# the dictionary: 
def distance_to_all(word):
    def dist_to_word(w):
        return distance(w, word)
    distances = words[0].apply(dist_to_word)
    return distances

In [ ]:
distance_to_all("wprd")

Let's make a new dataframe with the words and the distances to "wprd":

In [ ]:
wprd_dist = words.copy()
wprd_dist["distance"] = distance_to_all("wprd")
wprd_dist

Let us examine the histogram of distances.  All the words in the dictionary are in there.

In [ ]:
wprd_dist.distance.value_counts()

This ordering of the histogram is not ideal for understanding, let us sort by distance:

In [ ]:
wprd_dist.distance.value_counts().sort_index()

From the table, which gives the values of edit distance as the index and the number of dictionary words at each distance from our target word as values, we see there are three dictionary words at distance 1, and 147 words at edit distance 2. 

In [ ]:
wprd_dist.query("distance < 2")

In [ ]:
wprd_dist.query("distance == 2")

This certainly gives us a starting point (or at least a partly ordered list) of plausible words.  

## Diffs

An alignment-based representation of the changes to a document is sometimes an effective way of learning what has changed; algorithms that will compare documents, find the minimum number of edits and show what the edits are with highlighting can be helpful when examining changes to documents or code.